# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = os.path.join('..', 'WeatherPy', 'output_data', 'cities.csv')
df = pd.read_csv(city_data)
df.dropna(inplace=True)
df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Newberry,1,US,1595266632,50,34.27,-81.62,100.00,8.05
1,1,Kapaa,40,US,1595266477,69,22.08,-159.32,80.60,14.99
2,2,Ojinaga,1,MX,1595266633,33,29.57,-104.42,89.60,5.82
3,3,Myaundzha,31,RU,1595266633,70,63.05,147.19,53.26,3.56
4,4,Butaritari,90,KI,1595266478,67,3.07,172.79,82.78,6.58
...,...,...,...,...,...,...,...,...,...,...
571,571,Conde,75,BR,1595266781,83,-7.26,-34.91,78.80,16.11
572,572,Sevierville,1,US,1595266719,44,35.87,-83.56,93.20,5.82
573,573,Kumul,100,CN,1595266782,18,42.80,93.45,89.24,10.20
574,574,Ponta Delgada,75,PT,1595266782,56,37.73,-25.67,73.40,9.17


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [10]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = df[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
df.dropna(inplace=True)
humidity = df.Humidity.astype(float)
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
ideal = df[(df['Max Temp'] <= 85) & (df['Max Temp'] >= 75) & (df['Wind Speed'] <= 10) & (df['Cloudiness'] == 0)].reset_index(drop=True)
#rubric states the dataframe must have 10 or less rows,so I designed the following code to slice the first 10 rows.
if len(ideal) > 10:
    ideal = ideal.iloc[:10]
ideal.dropna(inplace=True)
ideal

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,105,Erzin,0,TR,1595266662,79,36.96,36.20,82.99,1.99
1,195,Florence,0,IT,1595266450,40,43.77,11.25,84.00,7.20
2,203,Hirara,0,JP,1595266688,79,24.80,125.28,84.63,9.80
3,205,Santa Isabel,0,BR,1595266540,36,-23.32,-46.22,78.80,9.17
4,283,Kiryat Gat,0,IL,1595266707,64,31.61,34.76,82.99,1.01
5,344,Semnan,0,IR,1595266722,25,35.57,53.40,84.65,7.67
6,449,Mahon,0,ES,1595266748,61,39.89,4.27,80.60,9.17
7,455,Quryq,0,KZ,1595266750,35,43.20,51.65,83.25,4.99


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df = ideal
hotel_df['Hotel Name'] = ""
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

p = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}



In [15]:
for index, row in hotel_df.iterrows():
    coordinates = f"{row['Lat']}, {row['Lng']}"
    p['location'] = coordinates
    print(f'Retrieving results for index {index}: {coordinates}.')
    r = requests.get(base_url, params=p).json()
    results = r['results']
    try:
        print(f"\tThe closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
    except (KeyError, IndexError):
        print('\tMissing data...skipping')
    print("------------")

Retrieving results for index 0: 36.96, 36.2.
	The closest hotel is Artemis Otel.
------------
Retrieving results for index 1: 43.77, 11.25.
	The closest hotel is Italiana Hotels Florence.
------------
Retrieving results for index 2: 24.8, 125.28.
	The closest hotel is Hotel Atoll Emerald Miyakojima.
------------
Retrieving results for index 3: -23.32, -46.22.
	The closest hotel is Chácara Sombra do Altíssimo.
------------
Retrieving results for index 4: 31.61, 34.76.
	The closest hotel is רגע בנחת.
------------
Retrieving results for index 5: 35.57, 53.4.
	The closest hotel is Semnan Tourist Hotel.
------------
Retrieving results for index 6: 39.89, 4.27.
	The closest hotel is ARTIEM Capri.
------------
Retrieving results for index 7: 43.2, 51.65.
	The closest hotel is Gostinitsa Kuryk - Otel' Dlya Vsekh Kategoriy Kliyentov ..
------------


In [16]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
0,105,Erzin,0,TR,1595266662,79,36.96,36.20,82.99,1.99,Artemis Otel
1,195,Florence,0,IT,1595266450,40,43.77,11.25,84.00,7.20,Italiana Hotels Florence
2,203,Hirara,0,JP,1595266688,79,24.80,125.28,84.63,9.80,Hotel Atoll Emerald Miyakojima
3,205,Santa Isabel,0,BR,1595266540,36,-23.32,-46.22,78.80,9.17,Chácara Sombra do Altíssimo
4,283,Kiryat Gat,0,IL,1595266707,64,31.61,34.76,82.99,1.01,רגע בנחת
5,344,Semnan,0,IR,1595266722,25,35.57,53.40,84.65,7.67,Semnan Tourist Hotel
6,449,Mahon,0,ES,1595266748,61,39.89,4.27,80.60,9.17,ARTIEM Capri
7,455,Quryq,0,KZ,1595266750,35,43.20,51.65,83.25,4.99,Gostinitsa Kuryk - Otel' Dlya Vsekh Kategoriy ...


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [18]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))